<a href="https://colab.research.google.com/github/Bibin32/Neural-Network-and-Hyper-Parameter-Tuning/blob/main/Hyperparameter_Tuning_Telescopic_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing

Load Dataset

In [23]:
df = pd.read_csv('magic_gamma_telescope04_.csv')

In [24]:
df.head()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [25]:
df['class'].nunique()

2

In [26]:
df1 = pd.get_dummies(df['class'],drop_first = 'True')

In [27]:
df1.head()

,h
0,0
1,0
2,0
3,0
4,0


In [28]:
df['class']=df1

In [29]:
df.head()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,0
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,0
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,0
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,0
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,0


In [30]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.25)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

11412 train examples
3804 validation examples
3804 test examples


In [31]:
df1 = train.copy()

ds = tf.data.Dataset.from_tensor_slices((dict(df1))) #, df1.pop('class')))
for item in ds:
  print(item)

Streaming output truncated to the last 5000 lines.
{'flength': <tf.Tensor: shape=(), dtype=float64, numpy=142.5279>, 'fwidth': <tf.Tensor: shape=(), dtype=float64, numpy=61.8344>, 'fsize': <tf.Tensor: shape=(), dtype=float64, numpy=3.9882>, 'fconc': <tf.Tensor: shape=(), dtype=float64, numpy=0.0898>, 'fconc1': <tf.Tensor: shape=(), dtype=float64, numpy=0.0616>, 'fsym': <tf.Tensor: shape=(), dtype=float64, numpy=73.647>, 'fm3long': <tf.Tensor: shape=(), dtype=float64, numpy=-130.4584>, 'fm3trans': <tf.Tensor: shape=(), dtype=float64, numpy=41.6467>, 'falpha': <tf.Tensor: shape=(), dtype=float64, numpy=88.5362>, 'dist': <tf.Tensor: shape=(), dtype=float64, numpy=374.8083>, 'class': <tf.Tensor: shape=(), dtype=uint8, numpy=1>}
{'flength': <tf.Tensor: shape=(), dtype=float64, numpy=40.6449>, 'fwidth': <tf.Tensor: shape=(), dtype=float64, numpy=13.3364>, 'fsize': <tf.Tensor: shape=(), dtype=float64, numpy=2.5855>, 'fconc': <tf.Tensor: shape=(), dtype=float64, numpy=0.387>, 'fconc1': <tf.Ten

In [32]:
def df_to_dataset(df1, shuffle=True, batch_size=3):
  '''Creates a tf.data dataset from a Pandas Dataframe'''
  df1 = df1.copy()
  labels = df1 #.pop('class')
  ds = tf.data.Dataset.from_tensor_slices((dict(df1), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df1))
  #We'll also want to process this a batch-size at a time, so we can use the batch method.  
  ds = ds.batch(batch_size)
  #Prefetching allows us to optimise fetching of data and training
  ds = ds.prefetch(batch_size)
  return ds

In [33]:
train_ds = df_to_dataset(train)
train_ds

<PrefetchDataset element_spec=({'flength': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fwidth': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fsize': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fconc': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fconc1': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fsym': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fm3long': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fm3trans': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'falpha': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'dist': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'class': TensorSpec(shape=(None,), dtype=tf.uint8, name=None)}, TensorSpec(shape=(None, 11), dtype=tf.float64, name=None))>

In [34]:
[(train_features, label_batch)] = train_ds.take(1)
label_batch

<tf.Tensor: shape=(3, 11), dtype=float64, numpy=
array([[ 6.23965e+01,  2.17309e+01,  2.70630e+00,  3.12700e-01,
         1.60300e-01, -6.61492e+01,  4.22396e+01,  7.76420e+00,
         6.15500e+00,  2.92278e+02,  0.00000e+00],
       [ 8.52941e+01,  7.95291e+01,  3.44220e+00,  2.30900e-01,
         1.47600e-01,  1.65248e+01,  4.74467e+01,  7.71849e+01,
         8.92721e+01,  4.23917e+02,  1.00000e+00],
       [ 4.64391e+01,  1.09253e+01,  2.92300e+00,  2.90100e-01,
         1.51000e-01,  1.60993e+01,  3.39589e+01,  6.24010e+00,
         4.96350e+01,  2.22490e+02,  1.00000e+00]])>

In [35]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of flength:', train_features['flength'])
print('A batch of labels:', label_batch )

Every feature: ['flength', 'fwidth', 'fsize', 'fconc', 'fconc1', 'fsym', 'fm3long', 'fm3trans', 'falpha', 'dist', 'class']
A batch of flength: tf.Tensor([33.295  97.509  32.2869], shape=(3,), dtype=float64)
A batch of labels: tf.Tensor(
[[ 3.329500e+01  3.051500e+01  3.006700e+00  2.421000e-01  1.302000e-01
   1.325140e+01 -5.688000e+00  1.431040e+01  2.774100e+00  1.053283e+02
   1.000000e+00]
 [ 9.750900e+01  3.837840e+01  3.309600e+00  1.314000e-01  7.380000e-02
   2.588580e+01  8.836160e+01 -1.828420e+01  8.585900e+00  2.162930e+02
   0.000000e+00]
 [ 3.228690e+01  1.621170e+01  2.638500e+00  4.161000e-01  2.356000e-01
   1.310900e+01  2.463150e+01 -1.205960e+01  5.825900e+00  1.958120e+02
   0.000000e+00]], shape=(3, 11), dtype=float64)


In [36]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None)

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # A pre-processing / non-traininable layer. 
  normalizer.adapt(feature_ds)

  return normalizer

In [37]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply one-hot encoding to our indices and return this feature
  return lambda feature: encoder(index(feature))

In [38]:
batch_size = 64
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [39]:
all_inputs = []
encoded_features = []

# Numeric features.
numeric_columns = ['flength','fwidth','fsize','fconc','fconc1','fsym','fm3long','fm3trans','falpha','dist','class']
for column_name in numeric_columns:
  numeric_column = tf.keras.Input(shape=(1,), name=column_name)
  normalization_layer = get_normalization_layer(column_name, train_ds)
  encoded_numeric_column = normalization_layer(numeric_column)
  all_inputs.append(numeric_column)
  encoded_features.append(encoded_numeric_column)

# Categorical features encoded as string.
#categorical_columns = ['class']
#for column_name in categorical_columns:
  #categorical_column = tf.keras.Input(shape=(1,), name=column_name, dtype='string')
  #encoding_layer = get_category_encoding_layer(column_name, train_ds, dtype='string', max_tokens=5)
  #encoded_categorical_column = encoding_layer(categorical_column)
  #all_inputs.append(categorical_column)
  #encoded_features.append(encoded_categorical_column)

In [40]:
all_inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'flength')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fwidth')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fsize')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fconc')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fconc1')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fsym')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fm3long')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fm3trans')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'falpha')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dist')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'class')>]

In [41]:
encoded_features

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_11')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_12')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_13')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_14')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_15')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_16')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_17')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_18')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_19')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_20')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_21')>]

In [42]:
#compile
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(128, activation="relu")(all_features)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
output = tf.keras.layers.Dense(1)(x)

#Optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum)
#model.compile(metrics=['accuracy'])
#model.fit(xtrain,ytrain,epochs=epochs,batch_size=batch_size)
#test_loss, test_acc = model.evaluate(xtest,ytest,verbose=2)
#print("Test Accuracy:", test_acc)

model = tf.keras.Model(all_inputs, output)

model.compile(optimizer='Adam', loss = tf.keras.losses.MeanSquaredError,
              metrics=["accuracy"])

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=1)